# 回归问题

根据汽车属性参数预测油耗，可应用场景：预测气温、机器翻译、智能问答

## 1. 导入包，并打印版本信息

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt

boston_housing = tf.keras.datasets.boston_housing
models = tf.keras.models
layers = tf.keras.layers
activations = tf.keras.activations
optimizers = tf.keras.optimizers
losses = tf.keras.losses

print('tensorflow: ' + tf.__version__)
print('pandas: ' + pd.__version__)
print('numpy: ' + np.__version__)
print('matplotlib: ' + matplotlib.__version__)
print('seaborn: ' + sns.__version__)

## 2. 准备样本数据
汽车数据

| 栏位           | 说明                    |
| :-------------------| :--------------------------------|
| MPG            | 油耗， 需要预测的数据        |
| Cylinders        | 汽缸数                  |
| Displacement      | 排量                    |
| Horsepower       | 马力                    |
| Weight          | 总量                    |
| Acceleration      | 加速度                  |
| Model Year       | 生产年份                 |
| Origin          | 产地， 1: 美国，2: 欧洲，3: 日本 |


In [ ]:
dataset_path = tf.keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight', 'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(dataset_path, names=column_names, na_values = "?", comment='\t',sep=" ", skipinitialspace=True)
dataset = raw_dataset.copy()
dataset.head()

## 3. 预处理数据

### 3.1 清理无效数据

#### 3.1.1 查看无效数据

In [ ]:
dataset.isna().sum()

#### 3.1.2 清理无效数据

In [ ]:
dataset = dataset.dropna()

#### 3.1.3 再来看一下

In [ ]:
dataset.isna().sum()

### 3.2 处理类别栏位
`Origin`栏位通过值1，2，3来标识产地，这里把它转换成One-Hot模式，注意下面的输出，`Origin`栏位被移除，增加 `USA`、`Europe`、`Japan` 3个栏位

In [ ]:
origin = dataset.pop('Origin')
dataset['USA'] = (origin == 1)*1.0
dataset['Europe'] = (origin == 2)*1.0
dataset['Japan'] = (origin == 3)*1.0
dataset.tail()

### 3.3 将数据分离成训练集和测试集

In [ ]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
print('dataset.shape: ' + str(dataset.shape))
print('train_dataset.shape: ' + str(train_dataset.shape))
print('test_dataset.shape： ' + str(test_dataset.shape))

### 3.4 看一下数据分布
通俗一点：看点是不是能聚在一起
[seaborn.pairplot文档说明](https://seaborn.pydata.org/generated/seaborn.pairplot.html#seaborn.pairplot)

#### 3.4.1 用图来查看数据分布

In [ ]:
sns.pairplot(train_dataset[["MPG", "Cylinders", "Displacement", "Weight", "Acceleration"]], diag_kind="kde")
plt.show()

#### 3.4.2 使用Pandas查看数据分布

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()
print(train_stats)

### 3.5 分别从训练数据和测试数据中分理出标签数据

In [ ]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

### 3.6 数据标准化
NOTE: std决定是不是需要标准化，这里使用Pandas处理，也可以使用numpy处理
```python
normed_train_data = train_dataset.to_numpy()
mean = normed_train_data.mean(axis=0)
normed_train_data -= mean
std = normed_train_data.std(axis=0)
normed_train_data /= std

normed_test_data = test_dataset.to_numpy()
normed_test_data -=mean
normed_test_data /=std

```

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

### 3.7 再来看一下标准化后的数据

In [ ]:
train_stats = normed_train_data.describe()
train_stats = train_stats.transpose()
print(train_stats)

## 4.  创建模型

In [ ]:
model = models.Sequential(
    [
        # NOTE： 飞驰人生：一顿操作猛如虎，定睛一看原地杵。
        layers.Dense(64, activation=activations.relu, input_shape=(normed_train_data.shape[1],)),
        layers.Dense(64, activation=activations.relu),
        layers.Dense(1)
    ]
)

model.compile(
    loss=losses.mean_squared_error,
    optimizer=optimizers.RMSprop(0.001),
    metrics=['mean_absolute_error', 'mean_squared_error']
)

model.summary()


## 5. 训练模型

In [ ]:
history = model.fit(
    normed_train_data,
    train_labels,
    epochs=1000, 
    validation_split = 0.2
)

## 6. 查看训练结果

In [ ]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure(figsize=(18, 10))
    plt.subplot(2, 1, 1)
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [MPG]')
    plt.plot(hist['epoch'], hist['mean_absolute_error'], label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_absolute_error'], label = 'Val Error')
    plt.ylim([0,5])
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$MPG^2$]')
    plt.plot(hist['epoch'], hist['mean_squared_error'], label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_squared_error'], label = 'Val Error')
    plt.ylim([0,20])
    plt.legend()
    plt.show()

plot_history(history)

## 7. 使用tf.keras.callbacks.EarlyStopping提前终止训练

NOTE: 自动化控制将是深度学习的一个重要发展方向

callbacks参数允许通过编程的方式干预学习过程，比如：
- 当发生过拟合的时候 `提前终止学习`、
- 学习过程分步 `保存模型`

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(normed_train_data, train_labels, epochs=1000, validation_split = 0.2, verbose=0, callbacks=[early_stop])
plot_history(history)

## 8. 使用测试集评估模型

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=0)
print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))

## 9. 在测试集上使用模型预测结果

使用plt图形化显示误差

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

plt.figure(figsize=(10, 10))
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
plt.plot([-100, 100], [-100, 100])

## 10. 看看误差值是否服从高斯正态分布

NOTE: 扩展一下 6 Sigma

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [MPG]")
plt.ylabel("Count")

<img src="./images/gaussian-distribution.jpg" />
<center>
    平均值为 μ 标准差为 σ 的正态分布
</center>